In [1]:
import time
from datetime import timedelta

import html
import re

import os
import ntpath

import subprocess as sp

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Enter full path to xml file

In [2]:
## Enter full path to xml file
#xml_file = 'Badges.xml'
#xml_file = '../../../../stackexchange_v2/workspace/input/Badges.xml'
xml_file = '../SO_data_dump/Posts.xml'

In [3]:
#xml_file=input()

In [4]:
def get_xml_filepath_wtout_ext(file_wt_full_path):
    #/a/b/c.xml => /a/b/c
    return os.path.splitext(file_wt_full_path)[0]

In [5]:
def get_xml_filename(file_wt_full_path):
    filepath_wout_ext = get_xml_filepath_wtout_ext(file_wt_full_path)
    #/a/b/c => c
    return ntpath.basename(filepath_wout_ext)


In [6]:
def get_filepath(file_wt_full_path):
    #/a/b/c.csv => /a/b
    return os.path.dirname(file_wt_full_path)

In [7]:
def get_file(file_wt_full_path):
    #/a/b/c.csv => c.csv
    return os.path.basename(file_wt_full_path)

## Read xml file in block sizes

In [8]:
## Read xml file in block sizes
bg = bd.read_text(xml_file, blocksize='10MB')

In [9]:
bg

dask.bag<bag-from-delayed, npartitions=9354>

In [10]:
bg.take(1)

('\ufeff<?xml version="1.0" encoding="utf-8"?>\n',)

In [11]:
bg.take(5)

('\ufeff<?xml version="1.0" encoding="utf-8"?>\n',
 '<posts>\n',
 '  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="742" ViewCount="61738" Body="&lt;p&gt;I want to use a &lt;code&gt;Track-Bar&lt;/code&gt; to change a &lt;code&gt;Form&lt;/code&gt;\'s opacity.&lt;/p&gt;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&lt;pre class=&quot;lang-cs prettyprint-override&quot;&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&lt;blockquote&gt;&#xA;&lt;pre class=&quot;lang-none prettyprint-override&quot;&gt;&lt;code&gt;Cannot implicitly convert type decimal to double&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&lt;/blockquote&gt;&#xA;&lt;p&gt;I have tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt;, but then the &lt;code&gt;Control&lt;/code&gt; doesn\'t work. This code worked fine in a p

## Replace the xml header with empty string '<xml ...>' => ''

In [12]:
## Replace the xml header with empty string '<xml ...>' => ''
bg = bg.map(lambda line: re.sub(r'\ufeff\s*<\s*\?xml.*\?>\n', '', line)) 

In [13]:
bg.take(5)

('',
 '<posts>\n',
 '  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="742" ViewCount="61738" Body="&lt;p&gt;I want to use a &lt;code&gt;Track-Bar&lt;/code&gt; to change a &lt;code&gt;Form&lt;/code&gt;\'s opacity.&lt;/p&gt;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&lt;pre class=&quot;lang-cs prettyprint-override&quot;&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&lt;blockquote&gt;&#xA;&lt;pre class=&quot;lang-none prettyprint-override&quot;&gt;&lt;code&gt;Cannot implicitly convert type decimal to double&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&lt;/blockquote&gt;&#xA;&lt;p&gt;I have tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt;, but then the &lt;code&gt;Control&lt;/code&gt; doesn\'t work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserI

## Get the filename without the xml extension 

In [14]:
## Get the filename without the xml extension 
filename = get_xml_filename(xml_file)
print(filename)

Posts


## Make the filename lower case to get a tag

In [15]:
## Make the filename lower case to get a tag
tag = filename.lower()
print(tag)

posts


## Drop wherever this tag occurs in the xml file

In [16]:
## Drop wherever this tag occurs in the xml file
bg = bg.map(lambda line: re.sub(r'\s*<{}\s*.*?>\n|</{}>'.format(tag, tag), '', line))

In [17]:
bg.take(5)

('',
 '',
 '  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="742" ViewCount="61738" Body="&lt;p&gt;I want to use a &lt;code&gt;Track-Bar&lt;/code&gt; to change a &lt;code&gt;Form&lt;/code&gt;\'s opacity.&lt;/p&gt;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&lt;pre class=&quot;lang-cs prettyprint-override&quot;&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&lt;blockquote&gt;&#xA;&lt;pre class=&quot;lang-none prettyprint-override&quot;&gt;&lt;code&gt;Cannot implicitly convert type decimal to double&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&lt;/blockquote&gt;&#xA;&lt;p&gt;I have tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt;, but then the &lt;code&gt;Control&lt;/code&gt; doesn\'t work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" Las

## Filter out just the rows

In [18]:
## Filter out just the rows
bg = bg.filter(lambda line: line.find('<row') >= 0)

In [19]:
bg.take(5)

('  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="742" ViewCount="61738" Body="&lt;p&gt;I want to use a &lt;code&gt;Track-Bar&lt;/code&gt; to change a &lt;code&gt;Form&lt;/code&gt;\'s opacity.&lt;/p&gt;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&lt;pre class=&quot;lang-cs prettyprint-override&quot;&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&lt;blockquote&gt;&#xA;&lt;pre class=&quot;lang-none prettyprint-override&quot;&gt;&lt;code&gt;Cannot implicitly convert type decimal to double&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&lt;/blockquote&gt;&#xA;&lt;p&gt;I have tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt;, but then the &lt;code&gt;Control&lt;/code&gt; doesn\'t work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUse

## Use ElementTree to get all the attrbutes in xml tags

In [20]:
# Add extra key value to dict if they do not exixt
def add_key_val_if_not_exist(_dict):
    _dict
    keys = [
        'Id',
        'PostTypeId',
        'AcceptedAnswerId',
        'ParentId', 
        'CreationDate',
        'DeletionDate',
        'Score',
        'ViewCount',
        'Body',
        'OwnerUserId',
        'OwnerDisplayName',
        'LastEditorUserId',
        'LastEditorDisplayName',
        'LastEditDate',
        'LastActivityDate', 
        'Title',
        'Tags', 
        'AnswerCount',
        'CommentCount',
        'FavoriteCount',
        'ClosedDate',
        'CommunityOwnedDate',
        'ContentLicense'
    ]
    value=''
    for key in keys:
        if key not in _dict:
            _dict[key] = value
    return _dict

In [21]:
#https://stackoverflow.com/questions/43921237/access-nested-children-in-xml-file-parsed-with-elementtree
def content(root):
    list_of_dicts = []
    for child in root.iter():
        # update dict with keys that do not exist
        _dict = add_key_val_if_not_exist(child.attrib)
        list_of_dicts.append(_dict)
    
    return list_of_dicts

In [22]:
## Use ElementTree to get all the attrbutes in xml tags
#bg = bg.map(lambda row: et.fromstring(row).attrib)
bg = bg.map(lambda row: content(et.fromstring(row)))

In [23]:
bg.take(5)

([{'Id': '4',
   'PostTypeId': '1',
   'AcceptedAnswerId': '7',
   'CreationDate': '2008-07-31T21:42:52.667',
   'Score': '742',
   'ViewCount': '61738',
   'Body': '<p>I want to use a <code>Track-Bar</code> to change a <code>Form</code>\'s opacity.</p>\n<p>This is my code:</p>\n<pre class="lang-cs prettyprint-override"><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n<p>When I build the application, it gives the following error:</p>\n<blockquote>\n<pre class="lang-none prettyprint-override"><code>Cannot implicitly convert type decimal to double\n</code></pre>\n</blockquote>\n<p>I have tried using <code>trans</code> and <code>double</code>, but then the <code>Control</code> doesn\'t work. This code worked fine in a past VB.NET project.</p>\n',
   'OwnerUserId': '8',
   'LastEditorUserId': '3072350',
   'LastEditorDisplayName': 'Rich B',
   'LastEditDate': '2021-02-26T03:31:15.027',
   'LastActivityDate': '2021-11-15T21:15:29.713',
   'Title': 'How to

## Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

disabling the mechanism to write on disk when workers run out of memory,
memory, CPUs, maximum time and number of workers per SLURM job,
dask folders for log files and workers data.
We recommend putting the log folder and workers data folders in your /nesi/nobackup/<project_code> folder, most indicated for temporary files (see NeSI File Systems and Quotas).

All of these options can be set in configuration files, see Dask configuration and Dask jobqueue configuration for more information.

In [24]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-30:30",# walltime="0-00:30",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42296 instead
  warnings.warn(


In [25]:
#cluster.adapt(minimum_jobs=20, maximum_jobs=200)
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.170:33636 Dashboard: http://192.168.94.170:42296/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Convert Bags to Dataframes

In [33]:
#flatten
bg = bg.flatten()
#df = files_rows_bd.badge_bg(meta={'name':'str', 'line':'int', 'severity':'str', 'message':'str', 'source':'str'})

In [34]:
#convert from bags to dataframe
df = bg.to_dataframe()
#df = files_rows_bd.badge_bg(meta={'name':'str', 'line':'int', 'severity':'str', 'message':'str', 'source':'str'})

In [35]:
df

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,CommunityOwnedDate,ContentLicense,ParentId,DeletionDate,OwnerDisplayName,ClosedDate
npartitions=9354,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [36]:
df.columns

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'CreationDate', 'Score',
       'ViewCount', 'Body', 'OwnerUserId', 'LastEditorUserId',
       'LastEditorDisplayName', 'LastEditDate', 'LastActivityDate', 'Title',
       'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount',
       'CommunityOwnedDate', 'ContentLicense', 'ParentId', 'DeletionDate',
       'OwnerDisplayName', 'ClosedDate'],
      dtype='object')

In [30]:
df.npartitions

9354

## Make a folder in that directory

In [31]:
## Make a folder in that directory
#folder = get_xml_filepath_wtout_ext(xml_file).lower() # the challenge is that the whole path to lowercase
## I just want the last folder to be a lower case
path = get_filepath(xml_file)
# output: path/to/Post.xml => path/to 
folder =  '{}/{}'.format(path, filename.lower())
# output: path/to/Post.xml => path/to/post
mkdir_cmd = 'mkdir {}'.format(folder)

In [32]:
#folder = get_xml_filepath_wtout_ext(xml_file)
print(folder)

../SO_data_dump/posts


## Save files in that directory